In [1]:

%pylab inline

import math
import numpy as np
from scipy.sparse.linalg import inv
#from numpy.linalg import inv
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy import integrate
import sys
import matplotlib.pyplot as plt
sys.path.append('../../src/')
from pylab import *

import parameters as pam
import lattice as lat
import variational_space as vs
import hamiltonian as ham
import basis_change as basis
import get_state as getstate
import utility as util
import plotfig as fig
import ground_state as gs
import lanczos
import time
start_time = time.time()
M_PI = math.pi
                  
#####################################
def compute_Aw_main(A,epx,epy,tpd,tpp,pds,pdp,pps,ppp,Upp,Udif,Vpp,Upd,Updf,\
                    d_double,pUppx_double,pUppy_double,pUpp_double,pVpp_double,pUpdx_double,pUpdy_double,U, S_val, Sz_val, AorB_sym): 
    diff = round(abs(epx-epy)*0.5,3)
    if Norb==7:
        fname = 'epx'+str(epx)+'epy'+str(epy)+'_tpd'+str(tpd)+'_tpp'+str(tpp) \
                  +'_A'+str(A)+'_B'+str(pam.B)+'_C'+str(pam.C)+'_Upp'+str(Upp)+'_Vpp'+str(Vpp)+'_Upd'+str(Upd) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta)
        flowpeak = 'Norb'+str(Norb)+'_tpp'+str(tpp)+'_diff'+str(diff)+'_Upp'+str(Upp)+'_Udif'+str(Udif)+\
                '_Vpp'+str(Vpp)+'_Upd'+str(Upd)+'_Updf'+str(Updf)+'_Mc'+str(Mc)+'_eta'+str(eta)
    elif Norb==9 or Norb==10 or Norb==11:
        fname = 'epx'+str(epx)+'epy'+str(epy)+'_pdp'+str(pdp)+'_pps'+str(pps)+'_ppp'+str(ppp) \
                   +'_A'+str(B)+'_B'+str(pam.B)+'_C'+str(pam.C)+'_Upp'+str(Upp)+'_Vpp'+str(Vpp)+'_Upd'+str(Upd) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta)
        flowpeak = 'Norb'+str(Norb)+'_pps'+str(pps)+'_ppp'+str(ppp)+'_diff'+str(diff)+'_Upp'+str(Upp)+'_Udif'+str(Udif)\
                   +'_Vpp'+str(Vpp)+'_Upd'+str(Upd)+'_Updf'+str(Updf)+'_Mc'+str(Mc)+'_eta'+str(eta)
                
    w_vals = np.arange(pam.wmin, pam.wmax, pam.eta/4.0)
   
    Aw = np.zeros(len(w_vals))
    Aw_dd_total = np.zeros(len(w_vals))
    Aw_d8_total = np.zeros(len(w_vals))

    # set up H0
    if Norb==7:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,tpd,tpp,0,0,0,0)
    elif Norb==9 or Norb==11:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,0,0,pds,pdp,pps,ppp)
            
    T_pd   = ham.create_tpd_nn_matrix(VS,tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac)
    T_pp   = ham.create_tpp_nn_matrix(VS,tpp_nn_hop_fac)  
    Esite  = ham.create_edep_diag_matrix(VS,A,epx,epy)
    
    H0 = T_pd + T_pp + Esite  
    
    '''
    Below probably not necessary to do the rotation by multiplying U and U_d
    the basis_change.py is only for label the state as singlet or triplet
    and assign the interaction matrix
    '''
    if pam.if_H0_rotate_byU==1:
        H0_new = U_d.dot(H0.dot(U))
    
    clf()

    if Norb==7 or Norb==9 or Norb==10 or Norb==11:     
        Hint = ham.create_interaction_matrix_ALL_syms(VS,d_double,pUppx_double,pUppy_double,pUpp_double,pVpp_double,pUpdx_double,\
                                                      pUpdy_double,S_val, Sz_val,AorB_sym,A,Upp,Udif,Vpp,Upd,Updf)
        if pam.if_H0_rotate_byU==1:
            H = H0_new + Hint 
        else:
            H = H0 + Hint 
        H.tocsr()

        ####################################################################################
        # compute GS only for turning on full interactions
        if pam.if_get_ground_state==1:
            vals, vecs, wgt_d8, wgt_d9L, wgt_d10L2 = gs.get_ground_state(H, VS, S_val, Sz_val)
            if Norb==7:
#                 util.write_GS('Egs_'+flowpeak+'.txt',A,epx,epy,tpd,vals[0])
                util.write_GS_components('GS_weights_'+flowpeak+'.txt',A,epx,epy,tpd,wgt_d8, wgt_d9L, wgt_d10L2)
            elif Norb==9 or Norb==10 or Norb==11:
                util.write_GS2('Egs_'+flowpeak+'.txt',A,epx,epy,pds,pdp,vals[0])
                util.write_GS_components2('GS_weights_'+flowpeak+'.txt',A,epx,epy,pds,pdp,wgt_d8, wgt_d9L, wgt_d10L2)
            
        #########################################################################
        '''
        Compute A(w) for various states
        '''
        # compute d8
#         fig.compute_Aw_d8_sym(H, VS, d_double, S_val, Sz_val, AorB_sym, A, w_vals, "Aw_d8_sym_", fname)
        
        # compute d9L
#         b1L_state_indices, a1L_state_indices, b1L_state_labels, a1L_state_labels \
#                 = getstate.get_d9L_state_indices(VS, S_val, Sz_val)
#         fig.compute_Aw1(H, VS, w_vals, b1L_state_indices, b1L_state_labels, "Aw_b1L_", fname)
#         fig.compute_Aw1(H, VS, w_vals, a1L_state_indices, a1L_state_labels, "Aw_a1L_", fname)
        
#         # compute d10L2
#         d10L2_state_indices, d10L2_state_labels = getstate.get_d10L2_state_indices(VS, S_val, Sz_val)
#         fig.compute_Aw1(H, VS, w_vals, d10L2_state_indices, d10L2_state_labels, "Aw_d10L2_", fname)
        
##########################################################################
if __name__ == '__main__': 
    Mc  = pam.Mc
    print ('Mc=',Mc)

    Norb = pam.Norb
    eta  = pam.eta
    ed   = pam.ed

    As = pam.As
    B  = pam.B
    C  = pam.C
    
    # set up VS
    VS = vs.VariationalSpace(Mc)
    basis.count_VS(VS)
    
    d_double,pUppx_double,pUppy_double,pUpp_double, pVpp_double, pUpdx_double,pUpdy_double = ham.get_double_occu_list(VS)
    
    # change the basis for d_double states to be singlet/triplet
    if pam.basis_change_type=='all_states':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix(VS,d_double)
       
        if pam.if_print_VS_after_basis_change==1:
            basis.print_VS_after_basis_change(VS,S_val,Sz_val)
    elif pam.basis_change_type=='d_double':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix_d_double(VS,d_double)

    U_d = (U.conjugate()).transpose()
    # check if U if unitary
    #checkU_unitary(U,U_d)
    
    if Norb==7:
        
        for tpd in pam.tpds:
            for ep1 in pam.ep_avgs: 
                
                for ep2 in pam.ep_difs: 
                    
                    epx = ep1+ep2
                    epy = ep1-ep2
                    
                    for A in pam.As:
                        util.get_atomic_d8_energy(A,B,C)
                        for tpp in pam.tpps:
                            for Upp in pam.Upps:
                                Upp = round(Upp,3)
                                for Udif in pam.Udifs:
                                    Udif = round(Udif,3)
                                    for Vpp in pam.Vpps:
                                        Vpp = round(Vpp,3)
                                        
                                        for Upd in pam.Upds:
                                            Upd = round(Upd,3)
                                            
                                            for Updf in pam.Updfs:
                                                
                                                Updf = round(Updf,3) 
                                                print ('===================================================')
                                                print ('A=',A, 'epx=', epx, 'epy=', epy, ' tpd=',tpd,' tpp=',tpp, \
                                                       ' Upp=',Upp,'Udif=',Udif,'Vpp=',Vpp,'Upd=',Upd,'Updf=',Updf)
                                                compute_Aw_main(A,epx,epy,tpd,tpp,0,0,0,0,Upp,Udif,Vpp,Upd,Updf,\
                                                                d_double,pUppx_double,pUppy_double,pUpp_double,\
                                                                pVpp_double,pUpdx_double,\
                                                                pUpdy_double,U, S_val, Sz_val, AorB_sym)

                                
    elif Norb==9 or Norb==10 or Norb==11:
        pps = pam.pps
        ppp = pam.ppp
        for ii in range(0,len(pam.pdps)):
            pds = pam.pdss[ii]
            pdp = pam.pdps[ii]
            for ep1 in pam.ep_avgs: 
                for ep2 in pam.ep_difs: 
                    epx = ep1+ep2
                    epy = ep1-ep2
                    for A in pam.As:
                        util.get_atomic_d8_energy(A,B,C)
                        for Upp in pam.Upps:
                            
                            Upp = round(Upp,3)
                            for Udif in pam.Udifs:
                                Udif = round(Udif,3)
                                for Vpp in pam.Vpps:
                                    Vpp = round(Vpp,3)
                                    for Upd in pam.Upds:
                                        Upd = round(Upd,3)
                                        for Updf in pam.Updfs:
                                            Updf = round(Updf,3)
                                            print ('===================================================')
                                            print ('A=',A, 'epx=', epx, 'epy=', epy,' pds=',pds,' pdp=',pdp,\
                                                   ' pps=',pps,' ppp=',ppp, ' Upp=',Upp, ' Udif=',Udif,' Vpp=',Vpp ,' Upd=',Upd,)
                                            compute_Aw_main(A,epx,epy,0,0,pds,pdp,pps,ppp,Upp,Udif,Vpp,Upd,Updf,\
                                                            d_double,pUpp_double,pVpp_double,pUpdx_double,pUpdy_double,\
                                                            U, S_val, Sz_val, AorB_sym)   


    print("--- %s seconds ---" % (time.time() - start_time))

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
Ni_orbs =  ['d3z2r2', 'dx2y2', 'dxy', 'dxz', 'dyz']
O1_orbs =  ['px']
O2_orbs =  ['py']
compute A(w) for symmetries =  ['1A1', '1B1', '3B1', '1A2', '3A2', '1E', '3E']
Mc= 10
VS.dim =  11025
No. of states with count_upup, count_updn, count_dnup, count_dndn: 0 5575 5450 0
len(d_list) 25
len(pUpp_list) 0
len(pUppx_list) 50
len(pUppy_list) 50
len(pVpp_list) 360
len(pUpdx_list) 20
len(pUpdy_list) 20
E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 3.5 epy= 3.5  tpd= 1.5  tpp= 0.55  Upp= 3.0 Udif= -1.0 Vpp= 0.0 Upd= 0.0 Updf= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-2.13695623 -0.7158706  -0.6026322  -0.57177634 -0.55168533 -0.55168533
 -0.54895946 -0.54777754 -0.42544282 -0.41643099]
eigenvalue =  -2.1

start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-2.13529469 -0.7158706  -0.6026322  -0.5717412  -0.55168533 -0.55168533
 -0.54873364 -0.54791336 -0.42541193 -0.41643099]
eigenvalue =  -2.1352946850180756
Compute the weights in GS (lowest Aw peak)
no e-h state  1680 px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.00961037704030951
no e-h state  2481 px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.02965703247747962
no e-h state  2499 py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.00733976859968017
no e-h state  2812 dx2y2 -2 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.006795614184369455
no e-h state  2822 dx2y2 -2 1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.006795614184369487
no e-h state  2892 dx2y2 -1 -2 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.006622877143269848
no e-h state  2902 dx2y2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.14939311027026517
no e-h state  2912 dx2y2 -1 2 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.006622877143269858
no e-h state  2992 dx2y2

no e-h state  5286 dx2y2 0 0 0 py 2 -1 0 S= 0 Sz= 0 , weight =  0.006764165245457848
no e-h state  5431 dx2y2 0 0 0 py 2 1 0 S= 0 Sz= 0 , weight =  0.006764165245457837
sumweight= 0.9575759621677065
A= 6.0 epx= 3.5 epy= 3.5  tpd= 1.5  tpp= 0.55  Upp= 3.0 Udif= -0.3 Vpp= 0.0 Upd= 0.0 Updf= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-2.13400013 -0.7158706  -0.6026322  -0.57171395 -0.55168533 -0.55168533
 -0.54846331 -0.5481141  -0.42538804 -0.41643099]
eigenvalue =  -2.1340001280393324
Compute the weights in GS (lowest Aw peak)
no e-h state  1680 px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.008876928714301133
no e-h state  2481 px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.02966300057357326
no e-h state  2499 py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.007909461646802644
no e-h state  2812 dx2y2 -2 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.006753024112242135
no e-h state  

sumweight= 0.9575742732427761
A= 6.0 epx= 3.5 epy= 3.5  tpd= 1.5  tpp= 0.55  Upp= 3.0 Udif= 0.1 Vpp= 0.0 Upd= 0.0 Updf= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-2.13374212 -0.7158706  -0.6026322  -0.57170853 -0.55168533 -0.55168533
 -0.54833992 -0.54822367 -0.4253833  -0.41643099]
eigenvalue =  -2.133742119478731
Compute the weights in GS (lowest Aw peak)
no e-h state  1680 px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.008215853738359844
no e-h state  2481 px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.029664183901186622
no e-h state  2499 py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.008537967796593915
no e-h state  2812 dx2y2 -2 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.006705114789507283
no e-h state  2822 dx2y2 -2 1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.006705114789507278
no e-h state  2892 dx2y2 -1 -2 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.0067297433694970404
no e-h state

sumweight= 0.9575759621677078
A= 6.0 epx= 3.5 epy= 3.5  tpd= 1.5  tpp= 0.55  Upp= 3.0 Udif= 0.5 Vpp= 0.0 Upd= 0.0 Updf= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-2.13451705 -0.7158706  -0.6026322  -0.57172482 -0.55168533 -0.55168533
 -0.54859435 -0.54801079 -0.42539756 -0.41643099]
eigenvalue =  -2.1345170465885492
Compute the weights in GS (lowest Aw peak)
no e-h state  1680 px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.007617753169185235
no e-h state  2481 px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.02966062365114559
no e-h state  2499 py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.009233956699040312
no e-h state  2812 dx2y2 -2 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.006651713019455739
no e-h state  2822 dx2y2 -2 1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.0066517130194557374
no e-h state  2892 dx2y2 -1 -2 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.006774976255486053
no e-h state

...

sumweight= 0.9896046646406848
A= 6.0 epx= 7.1 epy= 6.9  tpd= 1.5  tpp= 0.55  Upp= 3.0 Udif= -0.0 Vpp= 0.0 Upd= 0.0 Updf= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[1.32422321 1.67254787 2.36262617 2.37486401 2.71776947 2.96565308
 3.35673625 3.36201959 3.53485132 3.68247177]
eigenvalue =  1.3242232124832505
Compute the weights in GS (lowest Aw peak)
no e-h state  1680 px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.0037681786045305147
no e-h state  2481 px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.013097705041287905
no e-h state  2499 py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.004366537205099536
no e-h state  2812 dx2y2 -2 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.0037315069083245587
no e-h state  2822 dx2y2 -2 1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.003731506908324546
no e-h state  2892 dx2y2 -1 -2 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.003677153657407261
no e-h state  2902 d

start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[1.32375713 1.67254787 2.36262617 2.37486401 2.71765668 2.96565308
 3.35673625 3.36201959 3.53485132 3.68247177]
eigenvalue =  1.3237571293955983
Compute the weights in GS (lowest Aw peak)
no e-h state  1680 px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.003609093133099476
no e-h state  2481 px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.01309629208721204
no e-h state  2499 py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.004564750163847481
no e-h state  2812 dx2y2 -2 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.0037175008031715587
no e-h state  2822 dx2y2 -2 1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.0037175008031715643
no e-h state  2892 dx2y2 -1 -2 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.003688047602952516
no e-h state  2902 dx2y2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.14339668310165649
no e-h state  2912 dx2y2 -1 2 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.003688047602952514
no e-h state  2992 dx2y2 0 -1 0

no e-h state  5286 dx2y2 0 0 0 py 2 -1 0 S= 0 Sz= 0 , weight =  0.003703058442750815
no e-h state  5431 dx2y2 0 0 0 py 2 1 0 S= 0 Sz= 0 , weight =  0.003703058442750817
sumweight= 0.9894648440047413
A= 6.0 epx= 7.1 epy= 6.9  tpd= 1.5  tpp= 0.55  Upp= 3.0 Udif= 0.7 Vpp= 0.0 Upd= 0.0 Updf= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[1.32280011 1.67254787 2.36262617 2.37486401 2.71731428 2.96565308
 3.35673625 3.36201959 3.53485132 3.68247177]
eigenvalue =  1.3228001122034598
Compute the weights in GS (lowest Aw peak)
no e-h state  1680 px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.003410763208839327
no e-h state  2481 px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.013094726155958013
no e-h state  2499 py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.00484891217665411
no e-h state  2812 dx2y2 -2 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.003698142062377247
no e-h state  2822 dx2y2 -

sumweight= 0.9893905087519903
A= 6.0 epx= 7.1 epy= 6.9  tpd= 1.5  tpp= 0.55  Upp= 3.0 Udif= 1.0 Vpp= 0.0 Upd= 0.0 Updf= 0.0
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[1.32182685 1.67254787 2.36262617 2.37486401 2.71691199 2.96565308
 3.35673625 3.36201959 3.53485132 3.68247177]
eigenvalue =  1.3218268459523317
Compute the weights in GS (lowest Aw peak)
no e-h state  1680 px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.003271483303007537
no e-h state  2481 px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.013093778764170822
no e-h state  2499 py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.005078413824936274
no e-h state  2812 dx2y2 -2 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.00368306899463481
no e-h state  2822 dx2y2 -2 1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.003683068994634814
no e-h state  2892 dx2y2 -1 -2 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.003712141729817102
no e-h state  2902 dx2y2

<Figure size 640x480 with 0 Axes>